### Special thanks to [ilosvigil](https://www.kaggle.com/ilosvigil/scl2020-2-5a-model-private-lb-0-84923/data?scriptVersionId=37953296) with his complete source code available on [github](https://github.com/ilos-vigil/scl-2020-product-detection)

# Image Classification in a Nutshell

> This notebook source code based on [Ensemble, LAMB & GridMask on TPU](https://www.kaggle.com/ilosvigil/ensemble-lamb-gridmask-on-tpu) version 37 and 43

* Multimodal Deep Learning
    * Image
    * Text (from OCR)
* Dynamic image augmentation
    * GridMask (https://arxiv.org/abs/2001.04086) from [GridMask data augmentation with tensorflow](https://www.kaggle.com/xiejialun/gridmask-data-augmentation-with-tensorflow)
    * Rotate, shear, zoom, shift from [Rotation Augmentation GPU/TPU - [0.96+]](https://www.kaggle.com/cdeotte/rotation-augmentation-gpu-tpu-0-96)
    * [tf.image](https://www.tensorflow.org/api_docs/python/tf/image) functions
* TF-IDF word representation
    * L2 normalization
    * Sublinear Term Frequency
* EfficientNet B7 (https://arxiv.org/abs/1905.11946)
* Global Average Pooling (https://arxiv.org/abs/1312.4400)
* LAMB optimizer (https://arxiv.org/abs/1904.00962)
* MLP / Multi Layer Perceptron
* TTA / Test Time Augmentation
* TPU v3-8

# Library

In [ ]:
!pip install -q --upgrade efficientnet tensorflow_addons

In [ ]:
import math
import re
import random
import os
import itertools
import numpy as np
import pandas as pd
import matplotlib
import scipy
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow.keras.backend as K
import efficientnet.tfkeras as efn
import efficientnet
import sklearn

from matplotlib import pyplot as plt
from datetime import datetime

print(f'Numpy version : {np.__version__}')
print(f'Tensorflow version : {tf.__version__}')
print(f'Tensorflow Addons version : {tfa.__version__}')
print(f'EfficientNet (library) version : {efficientnet.__version__}')
print(f'Matplotlib version : {matplotlib.__version__}')
print(f'Scipy version : {scipy.__version__}')
print(f'Pandas version : {pd.__version__}')
print(f'Scikit-Learn version : {sklearn.__version__}')

## Set seed and Preconfig

In [ ]:
PRE_TRAINING_TIME_START = datetime.now()
AUTO = tf.data.experimental.AUTOTUNE

SEED = 42

os.environ['PYTHONHASHSEED']=str(SEED)
random.seed(SEED)
np.random.seed(SEED)

os.environ['TF_DETERMINISTIC_OPS']=str(SEED)
tf.random.set_seed(SEED)
# tf.config.threading.set_inter_op_parallelism_threads(1)
# tf.config.threading.set_intra_op_parallelism_threads(1)

## Detect TPU Availability

In [ ]:
# Detect hardware, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

print("REPLICAS: ", strategy.num_replicas_in_sync)

# Configuration

In [ ]:
from kaggle_datasets import KaggleDatasets
IMAGE_SIZE = (512, 512)

GCS_TRAIN_PATHS = [
    KaggleDatasets().get_gcs_path('tfrecords'),
    KaggleDatasets().get_gcs_path('tfrecords-2')
]
TRAINING_FILENAMES = []
for i in GCS_TRAIN_PATHS:
    TRAINING_FILENAMES.append(tf.io.gfile.glob(i + '/*.tfrecords'))
TRAINING_FILENAMES = list(itertools.chain.from_iterable(TRAINING_FILENAMES))

GCS_TEST_PATH = KaggleDatasets().get_gcs_path('tfrecords-3')
TEST_FILENAMES = tf.io.gfile.glob(GCS_TEST_PATH + '/*.tfrecords') # predictions on this dataset should be submitted for the competition

print(len(TRAINING_FILENAMES))
print(len(TEST_FILENAMES))

In [ ]:
EPOCHS = 12
DO_AUG = True

BATCH_SIZE = 256
current_epoch = 0 # used to determine augmentation rate
chance = 0

NUM_TRAINING_IMAGES = 105390
NUM_TEST_IMAGES = 12186
STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE

In [ ]:
CLASSES = [str(c).zfill(2) for c in range(0, 42)]

# TFRecord function

In [ ]:
def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=3)
    image = tf.cast(image, tf.float32) / 255.0  # convert image to floats in [0, 1] range
    image = tf.reshape(image, [*IMAGE_SIZE, 3]) # explicit size needed for TPU
    return image

def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "words": tf.io.FixedLenFeature([6633], tf.float32),  # shape [] means single element
        "label": tf.io.FixedLenFeature([], tf.int64),  # shape [] means single element
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    words = example['words']
    label = tf.cast(example['label'], tf.int32)
    
    return ((image, words), label) # returns a dataset of (image, label) pairs

def read_unlabeled_tfrecord(example):
    UNLABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "words": tf.io.FixedLenFeature([6633], tf.float32),  # shape [] means single element
        "filename": tf.io.FixedLenFeature([], tf.string),  # shape [] means single element
        # class is missing, this competitions's challenge is to predict flower classes for the test dataset
    }
    example = tf.io.parse_single_example(example, UNLABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    words = example['words']
    filename = example['filename']
    return ((image, words), filename) # returns a dataset of image(s)

def load_dataset(filenames, labeled=True, ordered=False):
    # Read from TFRecords. For optimal performance, reading from multiple files at once and
    # disregarding data order. Order does not matter since we will be shuffling the data anyway.

    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False # disable order, increase speed

    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO) # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order) # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(read_labeled_tfrecord if labeled else read_unlabeled_tfrecord, num_parallel_calls=AUTO)
    # returns a dataset of (image, label) pairs if labeled=True or (image, id) pairs if labeled=False
    return dataset

def get_training_dataset(do_aug=True):
    dataset = load_dataset(TRAINING_FILENAMES, labeled=True)
    if do_aug:
        dataset = dataset.map(image_augmentation, num_parallel_calls=AUTO)
    dataset = dataset.repeat() # the training dataset must repeat for several epochs
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

def get_test_dataset(ordered=False, tta=None):
    dataset = load_dataset(TEST_FILENAMES, labeled=False, ordered=ordered)
    if tta == 0:
        dataset = dataset.map(image_augmentation_tta_0, num_parallel_calls=AUTO)    
    elif tta == 1:
        dataset = dataset.map(image_augmentation_tta_1, num_parallel_calls=AUTO)
    elif tta == 2:
        dataset = dataset.map(image_augmentation_tta_2, num_parallel_calls=AUTO)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

# Image augmentation functions

* Chance of applying image augmentation based on training epoch with this formula
* `maxval` for most augmentation is 11

$$
\text{chance} =
\begin{cases}
    0 & \text{epoch} < 2 \\
    epoch - 1 & \text{epoch} < 9 \\
    8 & \text{epoch} \geq 9
\end{cases}
$$

$$
\text{chance'} = \frac{\text{chance}}{\text{maxval}}
$$



| Function   | Sub-chance | Range                                    |
| ---------- | ---------- | ---------------------------------------- |
| Flip       | -          | Left to right                            |
| Brightness | -          | 0.9 to 1.1                               |
| Contrast   | -          | 0.9 to 1.1                               |
| Saturation | -          | 0.95 to 1.05                             |
| Hue        | -          | 0.95 to 1.05                             |
| Rotate     | 50%        | 17 degrees * random normal distribution  |
| Shear      | 50%        | 5.5 degrees * random normal distribution |
| Zoom Out   | 33%        | 1.0 - (random normal distribution / 8.5) |
| Shift      | 33%        | 18 pixel * random normal distribution    |
| GridMask   | -          | d (square size) : 112 - 352 px           |
|            |            | r (keep ratio) : 0.4                     |
|            |            | 45 degress * random normal distribution  |

## Rotate, shear, zoom, shift

In [ ]:
@tf.function
def get_mat(rotation, shear, height_zoom, width_zoom, height_shift, width_shift):
    # returns 3x3 transformmatrix which transforms indicies
        
    # CONVERT DEGREES TO RADIANS
    rotation = math.pi * rotation / 180.
    shear = math.pi * shear / 180.
    
    # ROTATION MATRIX
    c1 = tf.math.cos(rotation)
    s1 = tf.math.sin(rotation)
    one = tf.constant([1],dtype='float32')
    zero = tf.constant([0],dtype='float32')
    rotation_matrix = tf.reshape( tf.concat([c1,s1,zero, -s1,c1,zero, zero,zero,one],axis=0),[3,3] )
        
    # SHEAR MATRIX
    c2 = tf.math.cos(shear)
    s2 = tf.math.sin(shear)
    shear_matrix = tf.reshape( tf.concat([one,s2,zero, zero,c2,zero, zero,zero,one],axis=0),[3,3] )    
    
    # ZOOM MATRIX
    zoom_matrix = tf.reshape( tf.concat([one/height_zoom,zero,zero, zero,one/width_zoom,zero, zero,zero,one],axis=0),[3,3] )
    
    # SHIFT MATRIX
    shift_matrix = tf.reshape( tf.concat([one,zero,height_shift, zero,one,width_shift, zero,zero,one],axis=0),[3,3] )
    
    return K.dot(K.dot(rotation_matrix, shear_matrix), K.dot(zoom_matrix, shift_matrix))


@tf.function
def transform(image):
    # input image - is one image of size [dim,dim,3] not a batch of [b,dim,dim,3]
    # output - image randomly rotated, sheared, zoomed, and shifted
    DIM = IMAGE_SIZE[0]
    XDIM = DIM%2 #fix for size 331

    if tf.random.uniform(shape=[], minval=0, maxval=2, dtype=tf.int32, seed=SEED) == 0: # 50% chance
        rot = 17. * tf.random.normal([1],dtype='float32')
    else:
        rot = tf.constant([0],dtype='float32')
    
    if tf.random.uniform(shape=[], minval=0, maxval=2, dtype=tf.int32, seed=SEED) == 0: # 50% chance
        shr = 5.5 * tf.random.normal([1],dtype='float32') 
    else:
        shr = tf.constant([0],dtype='float32')
    
    if tf.random.uniform(shape=[], minval=0, maxval=3, dtype=tf.int32, seed=SEED) == 0: # 33% chance
        h_zoom = tf.random.normal([1],dtype='float32')/8.5
        if h_zoom > 0:
            h_zoom = 1.0 + h_zoom * -1
        else:
            h_zoom = 1.0 + h_zoom
    else:
        h_zoom = tf.constant([1],dtype='float32')
    
    if tf.random.uniform(shape=[], minval=0, maxval=3, dtype=tf.int32, seed=SEED) == 0: # 33% chance
        w_zoom = tf.random.normal([1],dtype='float32')/8.5
        if w_zoom > 0:
            w_zoom = 1.0 + w_zoom * -1
        else:
            w_zoom = 1.0 + w_zoom
    else:
        w_zoom = tf.constant([1],dtype='float32')
    
    if tf.random.uniform(shape=[], minval=0, maxval=3, dtype=tf.int32, seed=SEED) == 0: # 33% chance
        h_shift = 18. * tf.random.normal([1],dtype='float32') 
    else:
        h_shift = tf.constant([0],dtype='float32')
    
    if tf.random.uniform(shape=[], minval=0, maxval=3, dtype=tf.int32, seed=SEED) == 0: # 33% chance
        w_shift = 18. * tf.random.normal([1],dtype='float32') 
    else:
        w_shift = tf.constant([0],dtype='float32')
  
    # GET TRANSFORMATION MATRIX
    m = get_mat(rot,shr,h_zoom,w_zoom,h_shift,w_shift) 

    # LIST DESTINATION PIXEL INDICES
    x = tf.repeat( tf.range(DIM//2,-DIM//2,-1), DIM )
    y = tf.tile( tf.range(-DIM//2,DIM//2),[DIM] )
    z = tf.ones([DIM*DIM],dtype='int32')
    idx = tf.stack( [x,y,z] )
    
    # ROTATE DESTINATION PIXELS ONTO ORIGIN PIXELS
    idx2 = K.dot(m,tf.cast(idx,dtype='float32'))
    idx2 = K.cast(idx2,dtype='int32')
    idx2 = K.clip(idx2,-DIM//2+XDIM+1,DIM//2)
    
    # FIND ORIGIN PIXEL VALUES           
    idx3 = tf.stack( [DIM//2-idx2[0,], DIM//2-1+idx2[1,]] )
    d = tf.gather_nd(image,tf.transpose(idx3))
        
    return tf.reshape(d,[DIM,DIM,3])

## GridMask

In [ ]:
@tf.function
def transform_grid_mark(image, inv_mat, image_shape):
    h, w, c = image_shape
    
    cx, cy = w//2, h//2

    new_xs = tf.repeat( tf.range(-cx, cx, 1), h)
    new_ys = tf.tile( tf.range(-cy, cy, 1), [w])
    new_zs = tf.ones([h*w], dtype=tf.int32)

    old_coords = tf.matmul(inv_mat, tf.cast(tf.stack([new_xs, new_ys, new_zs]), tf.float32))
    old_coords_x, old_coords_y = tf.round(old_coords[0, :] + tf.cast(w, tf.float32)//2.), tf.round(old_coords[1, :] + tf.cast(h, tf.float32)//2.)
    old_coords_x = tf.cast(old_coords_x, tf.int32)
    old_coords_y = tf.cast(old_coords_y, tf.int32)    

    clip_mask_x = tf.logical_or(old_coords_x<0, old_coords_x>w-1)
    clip_mask_y = tf.logical_or(old_coords_y<0, old_coords_y>h-1)
    clip_mask = tf.logical_or(clip_mask_x, clip_mask_y)

    old_coords_x = tf.boolean_mask(old_coords_x, tf.logical_not(clip_mask))
    old_coords_y = tf.boolean_mask(old_coords_y, tf.logical_not(clip_mask))
    new_coords_x = tf.boolean_mask(new_xs+cx, tf.logical_not(clip_mask))
    new_coords_y = tf.boolean_mask(new_ys+cy, tf.logical_not(clip_mask))

    old_coords = tf.cast(tf.stack([old_coords_y, old_coords_x]), tf.int32)
    new_coords = tf.cast(tf.stack([new_coords_y, new_coords_x]), tf.int64)
    rotated_image_values = tf.gather_nd(image, tf.transpose(old_coords))
    rotated_image_channel = list()
    for i in range(c):
        vals = rotated_image_values[:,i]
        sparse_channel = tf.SparseTensor(tf.transpose(new_coords), vals, [h, w])
        rotated_image_channel.append(tf.sparse.to_dense(sparse_channel, default_value=0, validate_indices=False))

    return tf.transpose(tf.stack(rotated_image_channel), [1,2,0])


@tf.function
def random_rotate(image, angle, image_shape):
    def get_rotation_mat_inv(angle):
          #transform to radian
        angle = math.pi * angle / 180

        cos_val = tf.math.cos(angle)
        sin_val = tf.math.sin(angle)
        one = tf.constant([1], tf.float32)
        zero = tf.constant([0], tf.float32)

        rot_mat_inv = tf.concat([cos_val, sin_val, zero,
                                     -sin_val, cos_val, zero,
                                     zero, zero, one], axis=0)
        rot_mat_inv = tf.reshape(rot_mat_inv, [3,3])

        return rot_mat_inv
    angle = float(angle) * tf.random.normal([1],dtype='float32')
    rot_mat_inv = get_rotation_mat_inv(angle)
    return transform_grid_mark(image, rot_mat_inv, image_shape)


@tf.function
def grid_mask():
    h = tf.constant(IMAGE_SIZE[0], dtype=tf.float32)
    w = tf.constant(IMAGE_SIZE[1], dtype=tf.float32)
    
    image_height, image_width = (h, w)
    d1 = 112 # 105
    d2 = 352 # 210
    rotate_angle = 45
    ratio = 0.6 # this is delete ratio, so keep ratio = 1 - delete ratio

    hh = tf.math.ceil(tf.math.sqrt(h*h+w*w))
    hh = tf.cast(hh, tf.int32)
    hh = hh+1 if hh%2==1 else hh
    d = tf.random.uniform(shape=[], minval=d1, maxval=d2, dtype=tf.int32)
    l = tf.cast(tf.cast(d,tf.float32)*ratio+0.5, tf.int32)

    st_h = tf.random.uniform(shape=[], minval=0, maxval=d, dtype=tf.int32)
    st_w = tf.random.uniform(shape=[], minval=0, maxval=d, dtype=tf.int32)

    y_ranges = tf.range(-1 * d + st_h, -1 * d + st_h + l)
    x_ranges = tf.range(-1 * d + st_w, -1 * d + st_w + l)

    for i in range(0, hh//d+1):
        s1 = i * d + st_h
        s2 = i * d + st_w
        y_ranges = tf.concat([y_ranges, tf.range(s1,s1+l)], axis=0)
        x_ranges = tf.concat([x_ranges, tf.range(s2,s2+l)], axis=0)

    x_clip_mask = tf.logical_or(x_ranges <0 , x_ranges > hh-1)
    y_clip_mask = tf.logical_or(y_ranges <0 , y_ranges > hh-1)
    clip_mask = tf.logical_or(x_clip_mask, y_clip_mask)

    x_ranges = tf.boolean_mask(x_ranges, tf.logical_not(clip_mask))
    y_ranges = tf.boolean_mask(y_ranges, tf.logical_not(clip_mask))

    hh_ranges = tf.tile(tf.range(0,hh), [tf.cast(tf.reduce_sum(tf.ones_like(x_ranges)), tf.int32)])
    x_ranges = tf.repeat(x_ranges, hh)
    y_ranges = tf.repeat(y_ranges, hh)

    y_hh_indices = tf.transpose(tf.stack([y_ranges, hh_ranges]))
    x_hh_indices = tf.transpose(tf.stack([hh_ranges, x_ranges]))

    y_mask_sparse = tf.SparseTensor(tf.cast(y_hh_indices, tf.int64),  tf.zeros_like(y_ranges), [hh, hh])
    y_mask = tf.sparse.to_dense(y_mask_sparse, 1, False)

    x_mask_sparse = tf.SparseTensor(tf.cast(x_hh_indices, tf.int64), tf.zeros_like(x_ranges), [hh, hh])
    x_mask = tf.sparse.to_dense(x_mask_sparse, 1, False)

    mask = tf.expand_dims( tf.clip_by_value(x_mask + y_mask, 0, 1), axis=-1)

    mask = random_rotate(mask, rotate_angle, [hh, hh, 1])
    mask = tf.image.crop_to_bounding_box(mask, (hh-tf.cast(h, tf.int32))//2, (hh-tf.cast(w, tf.int32))//2, tf.cast(image_height, tf.int32), tf.cast(image_width, tf.int32))

    return mask


@tf.function
def apply_grid_mask(image):
    mask = grid_mask()
    mask = tf.concat([mask, mask, mask], axis=-1)

    return image * tf.cast(mask, 'float32')

## Augmentation function & tf.image functions (flip, brightness, contrast, saturation, hue)

In [ ]:
# for train

@tf.function
def image_augmentation(iw, label):
    global current_epoch
    global chance
    
    image, words = iw

    if tf.random.uniform(shape=[], minval=0, maxval=11, dtype=tf.int32, seed=SEED) < chance:
        image = tf.image.flip_left_right(image)
    if tf.random.uniform(shape=[], minval=0, maxval=11, dtype=tf.int32, seed=SEED) < chance:
        image = tf.image.random_brightness(image, 0.1)
    if tf.random.uniform(shape=[], minval=0, maxval=11, dtype=tf.int32, seed=SEED) < chance:
        image = tf.image.random_contrast(image, 0.9, 1.1)
    if tf.random.uniform(shape=[], minval=0, maxval=11, dtype=tf.int32, seed=SEED) < chance:
        image = tf.image.random_saturation(image, 0.95, 1.05)
    if tf.random.uniform(shape=[], minval=0, maxval=11, dtype=tf.int32, seed=SEED) < chance:
        image = tf.image.random_hue(image, 0.05)

    if tf.random.uniform(shape=[], minval=0, maxval=11, dtype=tf.int32, seed=SEED) < chance:
        image = transform(image)

    if tf.random.uniform(shape=[], minval=0, maxval=10, dtype=tf.int32, seed=SEED) < chance:
        image = apply_grid_mask(image)

    return ((image, words), label)


In [ ]:
# for test / tta

# only flip left to right, brightness & contrast
@tf.function
def image_augmentation_tta_0(iw, filename):
    max_chance = 8
    
    image, words = iw

    if tf.random.uniform(shape=[], minval=0, maxval=11, dtype=tf.int32, seed=SEED) < max_chance:
        image = tf.image.flip_left_right(image)
    if tf.random.uniform(shape=[], minval=0, maxval=11, dtype=tf.int32, seed=SEED) < max_chance:
        image = tf.image.random_brightness(image, 0.1)
    if tf.random.uniform(shape=[], minval=0, maxval=11, dtype=tf.int32, seed=SEED) < max_chance:
        image = tf.image.random_contrast(image, 0.9, 1.1)

    return ((image, words), filename)

# all augmentation, except gridmask
@tf.function
def image_augmentation_tta_1(iw, filename):
    max_chance = 8
    
    image, words = iw

    if tf.random.uniform(shape=[], minval=0, maxval=11, dtype=tf.int32, seed=SEED) < max_chance:
        image = tf.image.flip_left_right(image)
    if tf.random.uniform(shape=[], minval=0, maxval=11, dtype=tf.int32, seed=SEED) < max_chance:
        image = tf.image.random_brightness(image, 0.1)
    if tf.random.uniform(shape=[], minval=0, maxval=11, dtype=tf.int32, seed=SEED) < max_chance:
        image = tf.image.random_contrast(image, 0.9, 1.1)
    if tf.random.uniform(shape=[], minval=0, maxval=11, dtype=tf.int32, seed=SEED) < max_chance:
        image = tf.image.random_saturation(image, 0.95, 1.05)
    if tf.random.uniform(shape=[], minval=0, maxval=11, dtype=tf.int32, seed=SEED) < max_chance:
        image = tf.image.random_hue(image, 0.05)

    if tf.random.uniform(shape=[], minval=0, maxval=11, dtype=tf.int32, seed=SEED) < max_chance:
        image = transform(image)

    return ((image, words), filename)

# all augmentation
@tf.function
def image_augmentation_tta_2(iw, filename):
    max_chance = 8
    
    image, words = iw

    if tf.random.uniform(shape=[], minval=0, maxval=11, dtype=tf.int32, seed=SEED) < max_chance:
        image = tf.image.flip_left_right(image)
    if tf.random.uniform(shape=[], minval=0, maxval=11, dtype=tf.int32, seed=SEED) < max_chance:
        image = tf.image.random_brightness(image, 0.1)
    if tf.random.uniform(shape=[], minval=0, maxval=11, dtype=tf.int32, seed=SEED) < max_chance:
        image = tf.image.random_contrast(image, 0.9, 1.1)
    if tf.random.uniform(shape=[], minval=0, maxval=11, dtype=tf.int32, seed=SEED) < max_chance:
        image = tf.image.random_saturation(image, 0.95, 1.05)
    if tf.random.uniform(shape=[], minval=0, maxval=11, dtype=tf.int32, seed=SEED) < max_chance:
        image = tf.image.random_hue(image, 0.05)

    if tf.random.uniform(shape=[], minval=0, maxval=11, dtype=tf.int32, seed=SEED) < max_chance:
        image = transform(image)

    if tf.random.uniform(shape=[], minval=0, maxval=10, dtype=tf.int32, seed=SEED) < max_chance:
        image = apply_grid_mask(image)

    return ((image, words), filename)

## Show augmentated image

In [ ]:
def show_augmented_image(same_image=True):
    row, col = 3, 5
    if same_image:
        all_elements = get_training_dataset(do_aug=False).unbatch()
        one_element = tf.data.Dataset.from_tensors( next(iter(all_elements)) )
        augmented_element = one_element.repeat().map(image_augmentation).batch(row*col)
        for iw, label in augmented_element:
            image, words = iw
            plt.figure(figsize=(15,int(15*row/col)))
            for j in range(row*col):
                plt.subplot(row,col,j+1)
                plt.axis('off')
                plt.imshow(image[j,])
            plt.suptitle(CLASSES[label[0]])
            plt.show()
            break
        
        del all_elements
        del one_element
        del augmented_element
    else:
        all_elements = get_training_dataset(do_aug=True).unbatch()
        augmented_element = all_elements.batch(row*col)

        for iw, label in augmented_element:
            image, words = iw
            plt.figure(figsize=(15,int(15*row/col)))
            for j in range(row*col):
                plt.subplot(row,col,j+1)
                plt.title(CLASSES[label[j]])
                plt.axis('off')
                plt.imshow(image[j,])
            plt.show()
            break
        
        del all_elements
        del augmented_element

In [ ]:
# run this cell again to see different image
# chance = 8
# show_augmented_image()
# chance = 0

In [ ]:
# run this cell again to see different batch of image
# chance = 8
# show_augmented_image(same_image=False)
# chance = 0

# Model function

In [ ]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, AveragePooling2D, GlobalAveragePooling2D, SpatialDropout2D, BatchNormalization, Activation, Concatenate

In [ ]:
def plt_lr(epoch_count):
    if epoch_count > 50:
        epoch_count = 50
    
    rng = [i for i in range(epoch_count)]

    plt.figure()
    y = [lrfn(x) for x in rng]
    plt.title(f'Learning rate schedule: {y[0]} to {y[epoch_count-1]}')
    plt.plot(rng, y)

def plt_acc(h):
    plt.figure()
    plt.plot(h.history["sparse_categorical_accuracy"])
    if 'val_sparse_categorical_accuracy' in h.history:
        plt.plot(h.history["val_sparse_categorical_accuracy"]) 
        plt.legend(["training","validation"])       
    else:
        plt.legend(["training"])
    plt.xlabel("epoch")
    plt.title("Sparse Categorical Accuracy")
    plt.show()

def plt_loss(h):
    plt.figure()
    plt.plot(h.history["loss"])
    if 'val_loss' in h.history:
        plt.plot(h.history["val_loss"]) 
        plt.legend(["training","validation"])       
    else:
        plt.legend(["training"])
    plt.legend(["training","validation"])
    plt.xlabel("epoch")
    plt.title("Loss")
    plt.show()

In [ ]:
class EpochCallback(tf.keras.callbacks.Callback):  
    def on_epoch_begin(self, epoch, logs=None):
        global current_epoch
        global chance

        current_epoch = epoch       
        if current_epoch < 2:
            chance = 0
        elif current_epoch < 9:
            chance = current_epoch - 1 # possible chance between 1 to 7
        else:
            chance = 8
        print(f'Epoch #{current_epoch}')
        print(datetime.now())

In [ ]:
es_val_acc = tf.keras.callbacks.EarlyStopping(
    monitor='val_sparse_categorical_accuracy', min_delta=0.001, patience=5, verbose=1, mode='auto',
    baseline=None, restore_best_weights=True
)

es_val_loss = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0.001, patience=5, verbose=1, mode='auto',
    baseline=None, restore_best_weights=True
)

es_acc = tf.keras.callbacks.EarlyStopping(
    monitor='sparse_categorical_accuracy', min_delta=0.001, patience=5, verbose=1, mode='auto',
    baseline=None, restore_best_weights=False
)

es_loss = tf.keras.callbacks.EarlyStopping(
    monitor='loss', min_delta=0.001, patience=5, verbose=1, mode='auto',
    baseline=None, restore_best_weights=False
)

epoch_cb = EpochCallback()

# Create model

In [ ]:
with strategy.scope():
    # phase 1
    efn7 = efn.EfficientNetB7(weights='noisy-student', include_top=False, input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
    for layer in efn7.layers:
        layer.trainable = True

    model_image = Sequential([
        efn7,
        GlobalAveragePooling2D(name='efficientnet-b7_gap'),
    ], name='b7-image')

    model_words = Sequential([
        Input((6633, ), name='mlp-words_input'),

        Dense(331, name='mlp-words_dense_1'),
        BatchNormalization(name='mlp-words_bn_1'),
        Activation('relu', name='mlp-words_act_1'),

        Dense(110, name='mlp-words_dense_2'),
        BatchNormalization(name='mlp-words_bn_2'),
        Activation('relu', name='mlp-words_act_2'),
    ], name='mlp-words')
    concatenate = Concatenate(name='concatenate')([model_image.output, model_words.output])
    output = Dense(len(CLASSES), activation='softmax', name='output')(concatenate)

    model = Model(inputs=[model_image.input, model_words.input], outputs=output)

    # phase 2
#     model = tf.keras.models.load_model('/kaggle/input/train-phase-1-085009/model.h5')
#     model.load_weights('/kaggle/input/train-phase-1-085009/model_weights.h5')

    model.compile(optimizer=tfa.optimizers.LAMB(0.01), loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

model.summary()



In [ ]:
print(f'Pre training time : {(datetime.now() - PRE_TRAINING_TIME_START).total_seconds()} seconds')

# Train model

In [ ]:
LR_START = 0.0005
LR_MAX = 0.001
LR_MIN = 0.00015
LR_RAMPUP_EPOCHS = 2
LR_SUSTAIN_EPOCHS = 0
LR_EXP_DECAY = 0.83

def lrfn(epoch):
    if epoch < LR_RAMPUP_EPOCHS:
        lr = (LR_MAX - LR_START) / LR_RAMPUP_EPOCHS * epoch + LR_START
    elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
        lr = LR_MAX
    else:
        lr = (LR_MAX - LR_MIN) * LR_EXP_DECAY**(epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS) + LR_MIN
    return lr
lr_schedule = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose = True)

plt_lr(EPOCHS)

In [ ]:
model.fit(
    get_training_dataset(do_aug=DO_AUG), steps_per_epoch=STEPS_PER_EPOCH,
    epochs=EPOCHS,
    callbacks=[es_acc, epoch_cb, lr_schedule], verbose=1
)

In [ ]:
h = model.history
plt_acc(h)
plt_loss(h)

In [ ]:
POST_TRAINING_TIME_START = datetime.now()

# Submit Result

In [ ]:
def test(tta=None):
    test_ds = get_test_dataset(ordered=True, tta=tta) # since we are splitting the dataset and iterating separately on images and ids, order matters.

    print(f'Computing predictions for TTA {tta}...')
    test_images_ds = test_ds.map(lambda iw, filename: [iw])
    model_pred = model.predict(test_images_ds)

    return model_pred

In [ ]:
model_pred = test(tta=None)
model_pred_tta_0 = test(tta=0)
model_pred_tta_1 = test(tta=1)
model_pred_tta_2 = test(tta=2)

## No TTA

In [ ]:
pred_plain = np.argmax(model_pred, axis=-1)

test_ds = get_test_dataset(ordered=True) # since we are splitting the dataset and iterating separately on images and ids, order matters.

test_ids_ds = test_ds.map(lambda iw, filename: filename).unbatch()
test_ids = next(iter(test_ids_ds.batch(pred_plain.shape[0]))).numpy().astype('U') # all in one batch

df_submission = pd.DataFrame({'filename': test_ids, 'category': pred_plain})
df_submission = df_submission.drop_duplicates()
df_submission['category'] = df_submission['category'].apply(lambda c: str(c).zfill(2))
df_submission

In [ ]:
df_submission.to_csv('submission.csv', index=False)
!head submission.csv

## No TTA + TTA type 0

In [ ]:
pred_tta_0 = np.mean(np.array([model_pred, model_pred_tta_0]), axis=0)
pred_tta_0 = np.argmax(pred_tta_0, axis=-1)

test_ds = get_test_dataset(ordered=True) # since we are splitting the dataset and iterating separately on images and ids, order matters.

test_ids_ds = test_ds.map(lambda iw, filename: filename).unbatch()
test_ids = next(iter(test_ids_ds.batch(pred_tta_0.shape[0]))).numpy().astype('U') # all in one batch

df_submission_tta_0 = pd.DataFrame({'filename': test_ids, 'category': pred_tta_0})
df_submission_tta_0 = df_submission_tta_0.drop_duplicates()
df_submission_tta_0['category'] = df_submission_tta_0['category'].apply(lambda c: str(c).zfill(2))
df_submission_tta_0

In [ ]:
df_submission_tta_0.to_csv('submission_tta_0.csv', index=False)
!head submission_tta_0.csv

## No TTA + TTA type 0 + TTA type 1 + TTA type 2

In [ ]:
pred_tta_all = np.mean(np.array([model_pred, model_pred_tta_0, model_pred_tta_1, model_pred_tta_2]), axis=0)
pred_tta_all = np.argmax(pred_tta_all, axis=-1)

test_ds = get_test_dataset(ordered=True) # since we are splitting the dataset and iterating separately on images and ids, order matters.

test_ids_ds = test_ds.map(lambda iw, filename: filename).unbatch()
test_ids = next(iter(test_ids_ds.batch(pred_tta_all.shape[0]))).numpy().astype('U') # all in one batch

df_submission_tta_all = pd.DataFrame({'filename': test_ids, 'category': pred_tta_all})
df_submission_tta_all = df_submission.drop_duplicates()
df_submission_tta_all['category'] = df_submission_tta_all['category'].apply(lambda c: str(c).zfill(2))
df_submission_tta_all

In [ ]:
df_submission_tta_all.to_csv('submission_tta_all.csv', index=False)
!head submission_tta_all.csv

## Save model

In [ ]:
model.save('model.h5')
model.save_weights('model_weights.h5')

In [ ]:
print(f'Post training time : {(datetime.now() - POST_TRAINING_TIME_START).total_seconds()} seconds')